# QGS model: MAOOAM

## Coupled ocean-atmosphere model version

This model version is a 2-layer channel QG atmosphere truncated at wavenumber 2 coupled, both by friction and heat exchange, to a shallow water ocean with 8 modes. 

More detail can be found in the article:
* Vannitsem, S., Demaeyer, J., De Cruz, L., & Ghil, M. (2015). *Low-frequency variability and heat transport in a low-order nonlinear coupled ocean–atmosphere model*. Physica D: Nonlinear Phenomena, **309**, 71-85. [doi:10.1016/j.physd.2015.07.006](https://doi.org/10.1016/j.physd.2015.07.006)
* De Cruz, L., Demaeyer, J. and Vannitsem, S. (2016). *The Modular Arbitrary-Order Ocean-Atmosphere Model: MAOOAM v1.0*, Geosci. Model Dev., **9**, 2793-2808. [doi:10.5194/gmd-9-2793-2016](https://doi.org/10.5194/gmd-9-2793-2016)

## Modules import

First, setting the path and loading of some modules

In [ ]:
import sys, os

In [ ]:
sys.path.extend([os.path.abspath('../../')])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
from matplotlib import rc
rc('font',**{'family':'serif','sans-serif':['Times'],'size':14})

Initializing the random number generator (for reproducibility). -- Disable if needed.

In [ ]:
np.random.seed(210217)

Importing the model's modules

In [ ]:
from qgs.params.params import QgParams
from qgs.integrators.integrator import RungeKuttaIntegrator
from qgs.functions.tendencies import create_tendencies
from qgs.plotting.util import std_plot

Importing the Lyapunovs Estimators

In [ ]:
from qgs.toolbox.lyapunov import LyapunovsEstimator, CovariantLyapunovsEstimator

and diagnostics

In [ ]:
from qgs.diagnostics.streamfunctions import MiddleAtmosphericStreamfunctionDiagnostic, OceanicLayerStreamfunctionDiagnostic
from qgs.diagnostics.temperatures import MiddleAtmosphericTemperatureDiagnostic, OceanicLayerTemperatureDiagnostic
from qgs.diagnostics.variables import VariablesDiagnostic, GeopotentialHeightDifferenceDiagnostic
from qgs.diagnostics.multi import MultiDiagnostic, FieldsDiagnosticsList

## Systems definition

General parameters

In [ ]:
# Time parameters
dt = 0.1
# Saving the model state n steps
write_steps = 100

number_of_trajectories = 1

Setting some model parameters

In [ ]:
# Model parameters instantiation with some non-default specs
model_parameters = QgParams()

# Mode truncation at the wavenumber 2 in both x and y spatial
# coordinates for the atmosphere
model_parameters.set_atmospheric_channel_fourier_modes(2, 2)
# Mode truncation at the wavenumber 2 in the x and at the 
# wavenumber 4 in the y spatial coordinates for the ocean
model_parameters.set_oceanic_basin_fourier_modes(2, 4)

In [ ]:
# Setting MAOOAM parameters according to the publication linked above
model_parameters.set_params({'kd': 0.0290, 'kdp': 0.0290, 'n': 1.5, 'r': 1.e-7,
                             'h': 136.5, 'd': 1.1e-7})
model_parameters.atemperature_params.set_params({'eps': 0.7, 'T0': 289.3,
                                                 'hlambda': 15.06})
model_parameters.gotemperature_params.set_params({'gamma': 5.6e8, 'T0': 301.46})

Setting the short-wave radiation component as in the publication above: $C_{\text{a},1}$ and $C_{\text{o},1}$ 


In [ ]:
model_parameters.atemperature_params.set_insolation(103.3333, 0)
model_parameters.gotemperature_params.set_insolation(310, 0)

Printing the model's parameters

In [ ]:
model_parameters.print_params()

Creating the tendencies function

In [ ]:
%%time
f, Df = create_tendencies(model_parameters)

## Time integration

Defining an integrator

In [ ]:
integrator = RungeKuttaIntegrator()
integrator.set_func(f)

Start on a random initial condition and integrate over a transient time to obtain an initial condition on the attractors 

In [ ]:
ic = np.random.rand(model_parameters.ndim)*0.01

In [ ]:
%%time
## Might take several minutes, depending on your cpu computational power.

for _ in range(20):
    integrator.integrate(0., 3000000., dt, ic=ic, write_steps=0)
    time, ic = integrator.get_trajectories()

Save the initial condition to reuse it later

In [ ]:
np.save('ic_saved.npy', ic)

Or load it

In [ ]:
# ic = np.load('ic_saved.npy')

Now integrate to obtain a trajectory on the attractor

In [ ]:
%%time
integrator.integrate(0., 3000000., dt, ic=ic, write_steps=write_steps)
reference_time, reference_traj = integrator.get_trajectories()

Plotting the result in 3D and 2D

In [ ]:
fig = plt.figure(figsize=(10, 8))
axi = fig.gca(projection='3d')

axi.scatter(reference_traj[21], reference_traj[29], reference_traj[0], s=0.2);

axi.set_xlabel('$'+model_parameters.latex_var_string[21]+'$')
axi.set_ylabel('$'+model_parameters.latex_var_string[29]+'$')
axi.set_zlabel('$'+model_parameters.latex_var_string[0]+'$');

In [ ]:
varx = 21
vary = 29
plt.figure(figsize=(10, 8))

plt.plot(reference_traj[varx], reference_traj[vary], marker='o', ms=0.1, ls='')

plt.xlabel('$'+model_parameters.latex_var_string[varx]+'$')
plt.ylabel('$'+model_parameters.latex_var_string[vary]+'$');

In [ ]:
var = 10
plt.figure(figsize=(10, 8))

plt.plot(model_parameters.dimensional_time*reference_time, reference_traj[var])

plt.xlabel('time (days)')
plt.ylabel('$'+model_parameters.latex_var_string[var]+'$');

In [ ]:
integrator.terminate()

## Lyapunov exponents and vectors estimation example

Backward Lyapunovs Estimation

In [ ]:
%%time

blvint = LyapunovsEstimator()

blvint.set_func(f, Df)
blvint.compute_lyapunovs(0., 1000000., 2000000., 0.1, 0.1, ic, write_steps=write_steps)
btl, btraj, bexp, bvec = blvint.get_lyapunovs()


In [ ]:
plt.figure(figsize=(15, 4))

mean_exp = np.mean(bexp, axis=-1)

x_pos = np.arange(1.,model_parameters.ndim+1,1)

plt.bar(x_pos, mean_exp)

plt.vlines(x_pos, -0.55, np.minimum(0.,mean_exp)-0.035, linestyles='dashdot', colors='tab:gray')

plt.xticks(x_pos, map(str,range(1, model_parameters.ndim+1,1)))
yt=[-0.5,-0.4,-0.3,-0.2,-0.1,0.,0.1]
plt.yticks(yt, map(str,yt))

plt.xlim(x_pos[0]-1., x_pos[-1]+1.)
plt.ylim(np.min(mean_exp)-0.1, np.max(mean_exp)+0.1)

plt.ylabel("Lyapunov exponent");
plt.xlabel("Index of the Lyapunov exponent");


In [ ]:
plt.figure(figsize=(10, 8))

plt.plot(model_parameters.dimensional_time*btl, bexp.T)

plt.xlabel('time (days)')

plt.title('Local Lyapunov exponents');

In [ ]:
plt.figure(figsize=(10, 8))

plt.plot(model_parameters.dimensional_time*btl, bvec[:, 0, :].T)

plt.xlabel('time (days)')

plt.title('Most unstable Lyapunov vector components');

In [ ]:
blvint.terminate()

Covariant Lyapunovs Estimation

In [ ]:
%%time

clvint = CovariantLyapunovsEstimator()

clvint.set_func(f, Df)
clvint.compute_clvs(0., 1000000., 2000000., 3000000., 0.1, 0.1, ic, write_steps=write_steps, method=1)
ctl, ctraj, cexp, cvec = clvint.get_clvs()


In [ ]:
plt.figure(figsize=(15, 4))

mean_exp = np.mean(cexp, axis=-1)

x_pos = np.arange(1.,model_parameters.ndim+1,1)

plt.bar(x_pos, mean_exp)

plt.vlines(x_pos, -0.55, np.minimum(0.,mean_exp)-0.035, linestyles='dashdot', colors='tab:gray')

plt.xticks(x_pos, map(str,range(1, model_parameters.ndim+1,1)))
yt=[-0.5,-0.4,-0.3,-0.2,-0.1,0.,0.1]
plt.yticks(yt, map(str,yt))

plt.xlim(x_pos[0]-1., x_pos[-1]+1.)
plt.ylim(np.min(mean_exp)-0.1, np.max(mean_exp)+0.1)

plt.ylabel("Lyapunov exponent");
plt.xlabel("Index of the Lyapunov exponent");


In [ ]:
plt.figure(figsize=(10, 8))

plt.plot(model_parameters.dimensional_time*ctl, cexp.T)

plt.xlabel('time (days)')

plt.title('Local Lyapunov exponents');

In [ ]:
plt.figure(figsize=(10, 8))

plt.plot(model_parameters.dimensional_time*ctl, cvec[:, 0, :].T)

plt.xlabel('time (days)')

plt.title('Most unstable Lyapunov vector components');

In [ ]:
clvint.terminate()

## Showing the resulting fields (animation)

This is an advanced feature showing the time evolution of diagnostic of the model. It shows simultaneously a scatter plot of the variables $\psi_{{\rm a}, 1}$, $\psi_{{\rm o}, 2}$ and $\theta_{{\rm o}, 2}$, with the corresponding atmospheric and oceanic streamfunctions and temperature at 500 hPa. In all the field plots, we also show the first covariant Lyapunov vector. Please read the documentation for more information.

Creating the diagnostics:

* For the 500hPa geopotential height:

In [ ]:
psi_a = MiddleAtmosphericStreamfunctionDiagnostic(model_parameters, geopotential=True)
psi_a_clyap = MiddleAtmosphericStreamfunctionDiagnostic(model_parameters, geopotential=True)

* For the 500hPa atmospheric temperature:

In [ ]:
theta_a = MiddleAtmosphericTemperatureDiagnostic(model_parameters)
theta_a_clyap = MiddleAtmosphericTemperatureDiagnostic(model_parameters)

* For the ocean streamfunction:

In [ ]:
psi_o = OceanicLayerStreamfunctionDiagnostic(model_parameters)
psi_o_clyap = OceanicLayerStreamfunctionDiagnostic(model_parameters)

* For the ocean temperature:

In [ ]:
theta_o = OceanicLayerTemperatureDiagnostic(model_parameters)
theta_o_clyap = OceanicLayerTemperatureDiagnostic(model_parameters)

* For the nondimensional variables $\psi_{{\rm a}, 1}$, $\psi_{{\rm o}, 2}$ and $\theta_{{\rm o}, 2}$:

In [ ]:
variable_nondim = VariablesDiagnostic([21, 29, 0], model_parameters, False)

* For the geopotential height difference between North and South:

In [ ]:
geopot_dim = GeopotentialHeightDifferenceDiagnostic([[[np.pi/model_parameters.scale_params.n, np.pi/4], [np.pi/model_parameters.scale_params.n, 3*np.pi/4]]],
                                                    model_parameters, True)


In [ ]:
# setting also the background
background = VariablesDiagnostic([21, 29, 0], model_parameters, False)
background.set_data(reference_time, reference_traj)

Selecting a subset of the data to plot:

In [ ]:
stride = 10
time = ctl[::stride]
traj = ctraj[:, ::stride]
lvec = cvec[:, 0, ::stride]

Loading the trajectory data into the diagnostics

In [ ]:
psi_a.set_data(time, traj)
psi_o.set_data(time, traj)
theta_a.set_data(time, traj)
theta_o.set_data(time, traj)

Loading the Lyapunov vectors data into the Lyapunov diagnostics

In [ ]:
psi_a_clyap.set_data(time, lvec)
psi_o_clyap.set_data(time, lvec)
theta_a_clyap.set_data(time, lvec)
theta_o_clyap.set_data(time, lvec)

Loading the data in a combined diagnostic (FieldsDiagnosticsList)

In [ ]:
psi_a_l = FieldsDiagnosticsList([psi_a, psi_a_clyap])
psi_o_l = FieldsDiagnosticsList([psi_o, psi_o_clyap])

theta_a_l = FieldsDiagnosticsList([theta_a, theta_a_clyap])
theta_o_l = FieldsDiagnosticsList([theta_o, theta_o_clyap])


Creating a multi diagnostic with all the diagnostics:

In [ ]:
m = MultiDiagnostic(2,3)
m.add_diagnostic(geopot_dim, 
                 diagnostic_kwargs={'style':'moving-timeserie'})
m.add_diagnostic(theta_a_l, 
                 diagnostic_kwargs={'style': ['image', 'contour'],
                                    'contour_labels': False,
                                    'show_time': True,
                                    'oro_kwargs': False}, 
                 plot_kwargs=[{'cmap': plt.get_cmap('gist_yarg')}, {'colors':'k'}])
m.add_diagnostic(theta_o_l, 
                 diagnostic_kwargs={'style': ['image', 'contour'],
                                    'contour_labels': False,
                                    'show_time': True,
                                    'oro_kwargs': False}, 
                 plot_kwargs=[{'cmap': plt.get_cmap('gist_yarg')}, {'colors':'k'}])
m.add_diagnostic(variable_nondim,
                 diagnostic_kwargs={'show_time':False,
                                    'background': background,
                                    'style':'3Dscatter'},
                 plot_kwargs={'ms': 0.2})
m.add_diagnostic(psi_a_l,
                 diagnostic_kwargs={'style': ['image', 'contour'],
                                    'contour_labels': False,
                                    'show_time': True,
                                    'oro_kwargs': False}, 
                 plot_kwargs=[{'cmap': plt.get_cmap('gist_yarg')}, {'colors':'k'}])
m.add_diagnostic(psi_o_l,
                 diagnostic_kwargs={'style': ['image', 'contour'],
                                    'contour_labels': False,
                                    'show_time': True,
                                    'oro_kwargs': False}, 
                 plot_kwargs=[{'cmap': plt.get_cmap('gist_yarg')}, {'colors':'k'}])

m.set_data(time, traj)

and show an interactive animation:

In [ ]:
rc('font',**{'family':'serif','sans-serif':['Times'],'size':12})
m.animate(figsize=(23,12))

or a movie (may takes some minutes to compute):

In [ ]:
%%time
rc('font',**{'family':'serif','sans-serif':['Times'],'size':12})
rc('animation', embed_limit=100.)
m.movie(figsize=(23.5,12), anim_kwargs={'interval': 100, 'frames':2000})